<a href="https://colab.research.google.com/github/sroy-10/genai/blob/main/LangChain/LangChain_v0.3_Structured_Output.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -r requirements.txt -q

In [ ]:
import os
import langchain
from dotenv import load_dotenv
load_dotenv("var.env")

print(langchain.__version__) # 0.3.25

In [ ]:
from langchain_openai import AzureChatOpenAI

os.environ["AZURE_OPENAI_API_KEY"] = os.environ["AZ_API_KEY"]
os.environ["AZURE_OPENAI_ENDPOINT"] = os.getenv("AZ_BASE") # https://YOUR-ENDPOINT.openai.azure.com/

model = AzureChatOpenAI(
    azure_deployment=os.getenv("AZ_MODEL"),
    api_version=os.getenv("AZ_API_VERSION"),
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)


## With Structurd Output

In [ ]:
from typing import TypedDict

class Review(TypedDict):
  summary: str
  sentiment: str

structured_model = model.with_structured_output(Review)

result = structured_model. invoke("""The hardware is great, but the software feels bloated. There are too many pre-installed apps that I can't remove. Also, the UI looks outdated compared to other brands. Hoping for a software update to fix this.""")

result

Output:

```
{'summary': 'The user appreciates the hardware quality but is dissatisfied with the software experience due to excessive pre-installed apps and an outdated user interface.',
 'sentiment': 'mixed'}
```

In [ ]:
from typing import Annotated

class Review(TypedDict):
  summary: Annotated[str, "A brief summary of the review"]
  sentiment: Annotated[str, "Return sentiment of the review, either 'negative', 'positive' or 'neutral'"]

structured_model = model.with_structured_output(Review)
result = structured_model. invoke("""The hardware is great, but the software feels bloated. There are too many pre-installed apps that I can't remove. Also, the UI looks outdated compared to other brands. Hoping for a software update to fix this.""")

result


Output:
```
{'summary': 'The user appreciates the hardware quality but is dissatisfied with the software experience due to bloatware and an outdated user interface.',
 'sentiment': 'mixed'}
```

The validation is not being performed properly. Sentiment should be either in `negative`, `positive` or `negative`. But still `mixed` is coming.

In [ ]:
from typing import Optional, Literal

review_text = """I recently upgraded to the Samsung Galaxy S24 Ultra, and I must say, it’s an absolute powerhouse! The Snapdragon 8 Gen 3 processor makes everything lightning fast—whether I’m gaming, multitasking, or editing photos. The 5000mAh battery easily lasts a full day even with heavy use, and the 45W fast charging is a lifesaver.

The S-Pen integration is a great touch for note-taking and quick sketches, though I don't use it often. What really blew me away is the 200MP camera—the night mode is stunning, capturing crisp, vibrant images even in low light. Zooming up to 100x actually works well for distant objects, but anything beyond 30x loses quality.

However, the weight and size make it a bit uncomfortable for one-handed use. Also, Samsung’s One UI still comes with bloatware—why do I need five different Samsung apps for things Google already provides? The $1,300 price tag is also a hard pill to swallow.

Pros:
Insanely powerful processor (great for gaming and productivity)
Stunning 200MP camera with incredible zoom capabilities
Long battery life with fast charging
S-Pen support is unique and useful

Review by Nitish Singh"""

# schema
class Review(TypedDict):

    key_themes: Annotated[list[str], "Write down all the key themes discussed in the review in a list"]
    summary: Annotated[str, "A brief summary of the review"]
    sentiment: Annotated[Literal["pos", "neg"], "Return sentiment of the review either negative, positive or neutral"]
    pros: Annotated[Optional[list[str]], "Write down all the pros inside a list"]
    cons: Annotated[Optional[list[str]], "Write down all the cons inside a list"]
    name: Annotated[Optional[str], "Write the name of the reviewer"]

structured_model = model.with_structured_output(Review)
structured_model.invoke(review_text)


Output:

```
{'key_themes': ['Performance',
  'Camera Quality',
  'Battery Life',
  'S-Pen Integration',
  'User Interface',
  'Price'],
 'summary': 'The Samsung Galaxy S24 Ultra is a high-performance smartphone that excels in processing power, camera capabilities, and battery life, but is slightly hindered by its size, weight, and pre-installed software.',
 'sentiment': 'pos',
 'pros': ['Insanely powerful Snapdragon 8 Gen 3 processor',
  'Stunning 200MP camera with excellent night mode and zoom capabilities',
  'Long-lasting 5000mAh battery with 45W fast charging',
  'Unique and useful S-Pen integration for note-taking and sketches'],
 'cons': ['Bulky and heavy, making one-handed use uncomfortable',
  "Presence of bloatware in Samsung's One UI",
  'High price point at $1,300'],
 'name': 'Samsung Galaxy S24 Ultra Review'}
```

## Pydantic

In [ ]:
!pip install pydantic -q

In [ ]:
from langchain_openai import ChatOpenAI
from typing import TypedDict, Annotated, Optional, Literal
from pydantic import BaseModel, Field

# schema
class Review(BaseModel):

    key_themes: list[str] = Field(description="Write down all the key themes discussed in the review in a list")
    summary: str = Field(description="A brief summary of the review")
    sentiment: Literal["pos", "neg"] = Field(description="Return sentiment of the review either negative, positive or neutral")
    pros: Optional[list[str]] = Field(default=None, description="Write down all the pros inside a list")
    cons: Optional[list[str]] = Field(default=None, description="Write down all the cons inside a list")
    name: Optional[str] = Field(default=None, description="Write the name of the reviewer")


structured_model = model.with_structured_output(Review)

result = structured_model.invoke("""I recently upgraded to the Samsung Galaxy S24 Ultra, and I must say, it’s an absolute powerhouse! The Snapdragon 8 Gen 3 processor makes everything lightning fast—whether I’m gaming, multitasking, or editing photos. The 5000mAh battery easily lasts a full day even with heavy use, and the 45W fast charging is a lifesaver.

The S-Pen integration is a great touch for note-taking and quick sketches, though I don't use it often. What really blew me away is the 200MP camera—the night mode is stunning, capturing crisp, vibrant images even in low light. Zooming up to 100x actually works well for distant objects, but anything beyond 30x loses quality.

However, the weight and size make it a bit uncomfortable for one-handed use. Also, Samsung’s One UI still comes with bloatware—why do I need five different Samsung apps for things Google already provides? The $1,300 price tag is also a hard pill to swallow.

Pros:
Insanely powerful processor (great for gaming and productivity)
Stunning 200MP camera with incredible zoom capabilities
Long battery life with fast charging
S-Pen support is unique and useful

Review by Nitish Singh
""")

print(result)

Output:

```
key_themes=['Performance', 'Camera quality', 'Battery life', 'S-Pen functionality', 'User interface', 'Price'] summary='The Samsung Galaxy S24 Ultra impresses with its powerful performance, exceptional camera capabilities, and long-lasting battery life, but falls short in terms of user interface and price.' sentiment='pos' pros=['Insanely powerful processor (great for gaming and productivity)', 'Stunning 200MP camera with incredible zoom capabilities', 'Long battery life with fast charging', 'S-Pen support is unique and useful'] cons=['Weight and size make it uncomfortable for one-handed use', 'Samsung’s One UI comes with bloatware', 'High price tag'] name='Nitish Singh'
```

## JSON Schema

In [ ]:
from langchain_openai import ChatOpenAI
from typing import TypedDict, Annotated, Optional, Literal
from pydantic import BaseModel, Field

# schema
json_schema = {
  "title": "Review",
  "type": "object",
  "properties": {
    "key_themes": {
      "type": "array",
      "items": {
        "type": "string"
      },
      "description": "Write down all the key themes discussed in the review in a list"
    },
    "summary": {
      "type": "string",
      "description": "A brief summary of the review"
    },
    "sentiment": {
      "type": "string",
      "enum": ["pos", "neg"],
      "description": "Return sentiment of the review either negative, positive or neutral"
    },
    "pros": {
      "type": ["array", "null"],
      "items": {
        "type": "string"
      },
      "description": "Write down all the pros inside a list"
    },
    "cons": {
      "type": ["array", "null"],
      "items": {
        "type": "string"
      },
      "description": "Write down all the cons inside a list"
    },
    "name": {
      "type": ["string", "null"],
      "description": "Write the name of the reviewer"
    }
  },
  "required": ["key_themes", "summary", "sentiment"]
}


structured_model = model.with_structured_output(json_schema)

result = structured_model.invoke("""I recently upgraded to the Samsung Galaxy S24 Ultra, and I must say, it’s an absolute powerhouse! The Snapdragon 8 Gen 3 processor makes everything lightning fast—whether I’m gaming, multitasking, or editing photos. The 5000mAh battery easily lasts a full day even with heavy use, and the 45W fast charging is a lifesaver.

The S-Pen integration is a great touch for note-taking and quick sketches, though I don't use it often. What really blew me away is the 200MP camera—the night mode is stunning, capturing crisp, vibrant images even in low light. Zooming up to 100x actually works well for distant objects, but anything beyond 30x loses quality.

However, the weight and size make it a bit uncomfortable for one-handed use. Also, Samsung’s One UI still comes with bloatware—why do I need five different Samsung apps for things Google already provides? The $1,300 price tag is also a hard pill to swallow.

Pros:
Insanely powerful processor (great for gaming and productivity)
Stunning 200MP camera with incredible zoom capabilities
Long battery life with fast charging
S-Pen support is unique and useful

Review by Nitish Singh
""")

print(result)

Output:
```
{'key_themes': ['Powerful Snapdragon 8 Gen 3 processor', 'Excellent performance for gaming and multitasking', 'Long-lasting 5000mAh battery', 'Fast 45W charging', 'Impressive 200MP camera with night mode', 'Effective zoom up to 30x', 'S-Pen integration', 'Device weight and size', 'Presence of bloatware in One UI', 'High price point'], 'summary': 'The Samsung Galaxy S24 Ultra impresses with its powerful performance, exceptional camera capabilities, and long battery life, though it is slightly hindered by its size, bloatware, and high price.', 'sentiment': 'pos', 'pros': ['Insanely powerful processor (great for gaming and productivity)', 'Stunning 200MP camera with incredible zoom capabilities', 'Long battery life with fast charging', 'S-Pen support is unique and useful'], 'cons': ['Weight and size make it uncomfortable for one-handed use', 'Presence of bloatware in One UI', 'High $1,300 price tag'], 'name': 'Nitish Singh'}
```